# Train Reenactment DeepFake using Pix2Pix GAN

- Pix2Pix is a paired image to image translation architecture
- In this notebook, we will leverage a pix2pix architecture to train for reenactment
- The GAN will be trained with input as facial landmarks and output as actual video frame

## Load Libraries & Utils

In [ ]:
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
from gan_utils import downsample_block, upsample_block, discriminator_block
from data_utils import plot_sample_images, batch_generator, get_samples

In [ ]:
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (8,8),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}

plt.rcParams.update(params)

In [ ]:
IMG_WIDTH = 256
IMG_HEIGHT = 256

## Prepare Generator
> U-Net Generator

The U-Net architecture uses skip connections to shuttle important features between the input and outputs. In case of pix2pix GAN, skip connections are added between every ith down-sampling and (n-i)th over-sampling layers, where n is the total number of layers in the generator. The skip connection leads to concatenation of all channels from the ith and (n-i)th layers.

In [ ]:
def build_generator(img_shape,channels=3,num_filters=64):
    # Image input
    input_layer = Input(shape=img_shape)

    # Downsampling
    down_sample_1 = downsample_block(input_layer, 
                                     num_filters, 
                                     batch_normalization=False)
    # rest of the down-sampling blocks have batch_normalization=true
    down_sample_2 = downsample_block(down_sample_1, num_filters*2)
    down_sample_3 = downsample_block(down_sample_2, num_filters*4)
    down_sample_4 = downsample_block(down_sample_3, num_filters*8)
    down_sample_5 = downsample_block(down_sample_4, num_filters*8)
    down_sample_6 = downsample_block(down_sample_5, num_filters*8)
    down_sample_7 = downsample_block(down_sample_6, num_filters*8)

    # Upsampling blocks with skip connections
    upsample_1 = upsample_block(down_sample_7, down_sample_6, num_filters*8)
    upsample_2 = upsample_block(upsample_1, down_sample_5, num_filters*8)
    upsample_3 = upsample_block(upsample_2, down_sample_4, num_filters*8)
    upsample_4 = upsample_block(upsample_3, down_sample_3, num_filters*8)
    upsample_5 = upsample_block(upsample_4, down_sample_2, num_filters*2)
    upsample_6 = upsample_block(upsample_5, down_sample_1, num_filters)

    upsample_7 = UpSampling2D(size=2)(upsample_6)
    output_img = Conv2D(channels, 
                        kernel_size=4, 
                        strides=1, 
                        padding='same', 
                        activation='tanh')(upsample_7)

    return Model(input_layer, output_img)

## Patch-GAN Discriminator
The authors for pix2pix propose a Patch-GAN setup for the discriminator which takes the required inputs and generates an output of size NxN. Each $x_{ij}$ element of the NxN output signifies whether the corresponding patch ij in the generated image is real or fake. Each output patch can be traced back to its initial input patch basis the effective receptive field for each of the layers.

In [ ]:
def build_discriminator(img_shape,num_filters=64):
    input_img = Input(shape=img_shape)
    cond_img = Input(shape=img_shape)

    # Concatenate input and conditioning image by channels 
    # as input for discriminator
    combined_input = Concatenate(axis=-1)([input_img, cond_img])

    # First discriminator block does not use batch_normalization
    disc_block_1 = discriminator_block(combined_input, 
                                       num_filters, 
                                       batch_normalization=False)
    disc_block_2 = discriminator_block(disc_block_1, num_filters*2)
    disc_block_3 = discriminator_block(disc_block_2, num_filters*4)
    disc_block_4 = discriminator_block(disc_block_3, num_filters*8)

    output = Conv2D(1, kernel_size=4, strides=1, padding='same')(disc_block_4)

    return Model([input_img, cond_img], output)

## Custom Training Loop

In [ ]:
def train(generator, 
          discriminator, 
          gan, 
          patch_gan_shape, 
          epochs,
          path='/content/data' ,
          batch_size=1, 
          sample_interval=50):
  
    # Ground truth shape/ Patch-GAN outputs
    real_y = np.ones((batch_size,) + patch_gan_shape)
    fake_y = np.zeros((batch_size,) + patch_gan_shape)

    for epoch in range(epochs):
        print("Epoch={}".format(epoch))
        for idx, (imgs_source, imgs_cond) in enumerate(batch_generator(path=path,
                                                                     batch_size=batch_size,
                                                                     img_res=[IMG_HEIGHT, IMG_WIDTH])):
            # train disriminator
            # generator generates outputs based on conditioned input images
            fake_imgs = generator.predict([imgs_cond])

            # calculate discriminator loss on real samples
            disc_loss_real = discriminator.train_on_batch([imgs_source, imgs_cond], 
                                                       real_y)
            # calculate discriminator loss on fake samples
            disc_loss_fake = discriminator.train_on_batch([fake_imgs, imgs_cond], 
                                                       fake_y)
            # overall discriminator loss
            discriminator_loss = 0.5 * np.add(disc_loss_real, disc_loss_fake)

            # train generator
            gen_loss = gan.train_on_batch([imgs_source, imgs_cond],
                                          [real_y, imgs_source])

            # training updates every 50 iterations
            if idx % 50 == 0:
                print ("[Epoch {}/{}] [Discriminator loss: {}, accuracy: {}] [Generato r loss: {}]".format(epoch, 
                                                                                                        epochs,
                                                                                                        discriminator_loss[0], 
                                                                                                        100*discriminator_loss[1],
                                                                                                        gen_loss[0]))

                # Plot and Save progress every few iterations
                if idx % sample_interval == 0:
                    plot_sample_images(generator=generator,
                                     path=path,
                                     epoch=epoch,
                                     batch_num=idx,
                                     output_dir='images')

## Train the Model

In [ ]:
discriminator = build_discriminator(img_shape=(IMG_HEIGHT,IMG_WIDTH,3),
                                    num_filters=64)
discriminator.compile(loss='mse',
                      optimizer=Adam(0.0002, 0.5),
                      metrics=['accuracy'])

In [ ]:
generator = build_generator(img_shape=(IMG_HEIGHT,IMG_WIDTH,3),
                            channels=3,
                            num_filters=64)

source_img = Input(shape=(IMG_HEIGHT,IMG_WIDTH,3))
cond_img = Input(shape=(IMG_HEIGHT,IMG_WIDTH,3))
fake_img = generator(cond_img)

discriminator.trainable = False
output = discriminator([fake_img, cond_img])

gan = Model(inputs=[source_img, cond_img], outputs=[output, fake_img])
gan.compile(loss=['mse', 'mae'],
            loss_weights=[1, 100],
            optimizer=Adam(0.0002, 0.5))

In [ ]:
# prepare patch size for our setup
patch = int(IMG_HEIGHT / 2**4)
patch_gan_shape = (patch, patch, 1)
print("Patch Shape={}".format(patch_gan_shape))

## Visualise Model

In [ ]:
tf.keras.utils.plot_model(generator)

## Training Begins!!!

In [ ]:
train(generator, discriminator, gan, patch_gan_shape, epochs=200, batch_size=1, sample_interval=200)

## Save the Model

In [ ]:

generator.save('gen_pix2pix_deepfake_obama.h5',save_format='h5')

In [ ]:
discriminator.save('dis_pix2pix_deepfake_obama.h5',save_format='h5')

In [ ]:

gan.save('gan_pix2pix_deepfake_obama.h5',save_format='h5')